# Model Architecture

In [ ]:
class Model(nn.Module):

    def __init__(self, model):
      super(Model, self).__init__()
      self.model = model 
      self.dropout = nn.Dropout(0.1)      
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768,512)      
      self.fc2 = nn.Linear(512,2)
      self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sent_id, mask):
      _, cls_hs = self.bert(sent_id, attention_mask=mask)
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc2(x)
      x = self.softmax(x)

      return x

Initilize the model, send the model to GPU

In [ ]:
device = torch.device("cuda")
model = Model(model)
model = model.to(device)

Initialize the optimizer

In [ ]:
from transformers import AdamW
# learning rate is 2e-5
optimizer = AdamW(model.parameters(), lr = 2e-5)

Initilize class weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# check if the classes are balances
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)
# and add class weights to model
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

Define cross entropy function and the number of training epochs

In [ ]:
# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# 100 training epochs
epochs = 100